<a href="https://colab.research.google.com/github/yutan0565/2020-2-OSSP-CP--YamiYami_Z_Z-5/blob/master/MobileNet_V3_Small_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 lib 불러오기

In [ ]:
# 모델 로드 
# top 그대로 유지해서 할꺼면, 같은 이미지에 대해서 테스트를 진행 해야함

In [ ]:
# 데이터 로드 , 데이터 기본 제공 말고, 추가적인 거는 내가 모양 맞춰서 따로 정제 해야함

In [ ]:
#https://github.com/Bao-Jiarong/VGG/blob/master/vgg.py
# 여기꺼 참고하면 좋을거 같음

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [3]:
print(raw_train_x.shape)
print(raw_train_y.shape)
print(raw_test_x.shape)
print(raw_test_y.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
# float을 넣을거면 0~1 사이 값으로 바꿔야함
train_x = raw_train_x/255
test_x = raw_test_x/255
train_y = raw_train_y
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

"""
def show_sample(i):
  print(raw_train_y[i][0], labels[raw_train_y[i][0]])
  plt.imshow(raw_train_x[i])
  plt.show()

for i in [2, 10, 12, 14]:
  show_sample(i)
"""

'\ndef show_sample(i):\n  print(raw_train_y[i][0], labels[raw_train_y[i][0]])\n  plt.imshow(raw_train_x[i])\n  plt.show()\n\nfor i in [2, 10, 12, 14]:\n  show_sample(i)\n'

In [5]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import models
from tensorflow.keras import layers

mobile = MobileNetV3Small(weights = 'imagenet', #  그냥 초기화 하는거면, 이거 지우기
            include_top = False,
            input_shape=(32,32,3))
mobile.summary()

# vgg conv 구조만 사용하고 마지막 FC layer는 다른거 사용
model = models.Sequential()
model.add(mobile)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(10, activation = "sigmoid"))
model.summary()

4349952/4334752 [==============================] - 0s 0us/step
Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 16, 16, 16)   432         ['rescaling[0][0]']              
                                                                                                  
 Conv/BatchNorm (BatchNormaliza  (None, 16, 16, 16)  64          ['Conv[0][0]']                   
 tion)              

In [6]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y, epochs=10, verbose=1, batch_size=128)

Epoch 1/10
391/391 [==============================] - 73s 172ms/step - loss: 1.8215 - accuracy: 0.3405
Epoch 2/10
391/391 [==============================] - 67s 171ms/step - loss: 1.4116 - accuracy: 0.4865
Epoch 3/10
391/391 [==============================] - 67s 171ms/step - loss: 1.2198 - accuracy: 0.5628
Epoch 4/10
391/391 [==============================] - 67s 171ms/step - loss: 1.1012 - accuracy: 0.6060
Epoch 5/10
391/391 [==============================] - 68s 175ms/step - loss: 1.0075 - accuracy: 0.6384
Epoch 6/10
391/391 [==============================] - 67s 172ms/step - loss: 0.9284 - accuracy: 0.6687
Epoch 7/10
391/391 [==============================] - 67s 171ms/step - loss: 0.8659 - accuracy: 0.6909
Epoch 8/10
391/391 [==============================] - 67s 171ms/step - loss: 0.7938 - accuracy: 0.7142
Epoch 9/10
391/391 [==============================] - 67s 171ms/step - loss: 0.7360 - accuracy: 0.7345
Epoch 10/10
391/391 [==============================] - 67s 171ms/step - l

In [7]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=1)

print(predicted)

313/313 [==============================] - 6s 15ms/step - loss: 5.2432 - accuracy: 0.1037
loss= 5.243160724639893
acc= 0.10369999706745148
[8 8 8 ... 8 8 8]


In [9]:
model.save('mobile_h5_model.h5')
model.save('mobile_another_type')

INFO:tensorflow:Assets written to: mobile_another_type/assets


INFO:tensorflow:Assets written to: mobile_another_type/assets


In [20]:
from tensorflow_model_optimization.python.core.quantization.keras import quantize
quantized_model = quantize.quantize_model(model)

ValueError: ignored

In [14]:
!pip install tensorflow_model_optimization

     |████████████████████████████████| 237 kB 29.8 MB/s 


In [16]:
from keras.models import load_model
import tensorflow_model_optimization as tfmot

test = load_model('mobile_h5_model.h5')
# 전체 양자화 해주기
quant_aware_model_all = tfmot.quantization.keras.quantize_model(test)
quant_aware_model_all.summary()

ValueError: ignored